In [1]:
import pandas as pd 
import numpy as np 
import pgeocode

## Zillow Home Values 

Zillow Home Value Index (ZHVI): A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range. The raw version of that mid-tier ZHVI time series is also available.

Zillow publishes top-tier ZHVI (`$`, typical value for homes within the 65th to 95th percentile range for a given region) and bottom-tier ZHVI (`$`, typical value for homes that fall within the 5th to 35th percentile range for a given region).

Zillow also publishes ZHVI for all single-family residences (`$`, typical value for all single-family homes in a given region), for condo/coops (`$`), for all homes with 1, 2, 3, 4 and 5+ bedrooms (`$`), and the ZHVI per square foot (`$`, typical value of all homes per square foot calculated by taking the estimated home value for each home in a given region and dividing it by the home’s square footage). Check out this overview of ZHVI and a deep-dive into its methodology. 

### What to call ZHVI

ZHVI represents the “typical” home value for a region. When referring to the ZHVI dollar amount, **it should be designated as the “typical home value for the region.”** An earlier version of ZHVI represented a median value, but this is no longer the case. Wording should be changed to reflect the new ZHVI, and should be “typical home value” — **it is NOT the “median home value”**.

### Datasets Resource
[Zillow Research](https://www.zillow.com/research/data/)

In [2]:
orlando_value = pd.read_csv("data/zhvi.csv")

In [3]:
# orlando_value = orlando_value[orlando_value['MsaName'] == "Orlando, FL"]

# let's look at the dataset
orlando_value.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,190910.0,...,1017983,1019412.0,1021002,1032348,1033124,1043246.0,1055207.0,1071538.0,1086779.0,1098866.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,263059.0,...,496294,498967.0,501470,503881,506039,508321.0,509785.0,511825.0,513977.0,516977.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,264057.0,...,1116560,1116672.0,1111868,1102444,1093311,1092823.0,1090455.0,1093248.0,1095014.0,1097064.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,201890.0,...,344118,346566.0,349659,354152,358645,363987.0,369797.0,376594.0,383990.0,392858.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,353798.0,...,633123,635804.0,638691,641464,643879,647013.0,649451.0,652624.0,654942.0,658724.0


In [4]:
# we only need zipcodes of the city of orlando, fl.
zip_code = [32801,32804,32789,32811,32835,32819,32839,32824,32827,32822,32829,32832,32808,32817,32828,32818,32821,32825,32826,32836,32837,32805,32806,32803,32814,32809,32831,32792,32812,32807]
city_orlando = orlando_value[orlando_value['RegionName'].isin(zip_code)]

city_orlando.reset_index(drop=True, inplace=True)
# city_orlando.drop(columns=['SizeRank','RegionID','RegionType','StateName','State','City','Metro','CountyName','1996-01-31', '1996-02-29', '2014-03', '2014-04', '2014-05',
#        '2014-06', '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12'], inplace=True)
city_orlando.drop(columns=['SizeRank','RegionID','RegionType','StateName','State','City','Metro','CountyName'], inplace=True)
city_orlando.drop(city_orlando.iloc[:,1:217], inplace=True, axis=1)
city_orlando.rename(columns={"RegionName":"zipcode"}, inplace=True)
city_orlando.fillna(0, inplace=True)

city_orlando.head()

C:\Users\aliso\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\aliso\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\aliso\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,zipcode,2014-01-31,2014-02-28,2014-03-31,2014-04-30,2014-05-31,2014-06-30,2014-07-31,2014-08-31,2014-09-30,...,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30
0,32822,110068.0,111724.0,113339.0,114830.0,115737.0,116191.0,116467.0,116702.0,116555.0,...,207860,209668.0,211229,213046,215063,217226.0,219868.0,222782.0,226231.0,230418.0
1,32828,230331.0,232203.0,233732.0,235912.0,237043.0,237422.0,237083.0,236557.0,235724.0,...,325149,327891.0,330530,333372,336709,340177.0,344563.0,349060.0,354359.0,360861.0
2,32792,173041.0,174204.0,174756.0,175777.0,176170.0,176968.0,177510.0,178528.0,178999.0,...,284987,287539.0,290751,293997,297043,299631.0,303114.0,306945.0,312076.0,318976.0
3,32825,169206.0,170885.0,172482.0,174286.0,175452.0,176173.0,176530.0,176474.0,175988.0,...,271575,273765.0,275891,278249,280900,283755.0,287298.0,291072.0,295576.0,301136.0
4,32839,105523.0,107278.0,109211.0,110894.0,111546.0,111669.0,111370.0,111327.0,110769.0,...,200477,202093.0,203596,205351,206813,208559.0,210735.0,213649.0,216987.0,221388.0


In [5]:
city_orlando.zipcode.unique()

array([32822, 32828, 32792, 32825, 32839, 32837, 32835, 32808, 32824,
       32818, 32811, 32812, 32807, 32817, 32806, 32819, 32789, 32821,
       32832, 32803, 32801, 32809, 32836, 32804, 32826, 32805, 32829,
       32827, 32814, 32831], dtype=int64)

In [6]:
# check columns
# city_orlando.columns

### Orlando 2015

In [7]:
# we need to analyze data after 2015
orlando_2015 = city_orlando[['zipcode', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31',
       '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30',
       '2015-12-31']]

orlando_2015.set_index("zipcode", inplace=True)

orlando_2015 = orlando_2015.append(orlando_2015.mean().round(2).rename("2015_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2015.rename(columns={"average":"2015_average"}, inplace=True)
# orlando_2015.fillna(0, inplace=True)
orlando_2015

,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2015_average
zipcode,,,,,,,,,,,,,
32822,118864.00,119266.00,119645.0,120335.0,121065.00,122185.00,123165.0,124436.0,125685.00,126971.00,127851.00,129479.00,123245.58
32828,238423.00,238779.00,239047.0,239822.0,240068.00,240948.00,242085.0,244041.0,246106.00,247670.00,248341.00,250104.00,242952.83
32792,181352.00,182154.00,182319.0,182915.0,184028.00,185362.00,187098.0,188456.0,189833.00,190422.00,191728.00,193749.00,186618.00
32825,179204.00,179908.00,180504.0,181397.0,181911.00,182873.00,184033.0,185725.0,187541.00,188980.00,190238.00,192332.00,184553.83
32839,114367.00,114580.00,114553.0,114919.0,115480.00,116372.00,117341.0,118419.0,119964.00,121332.00,121523.00,121869.00,117559.92
32837,203150.00,203413.00,203525.0,204593.0,205435.00,206454.00,207545.0,209030.0,211061.00,212753.00,213637.00,215399.00,207999.58
32835,162464.00,162982.00,163681.0,164677.0,165365.00,166292.00,167199.0,168541.0,169981.00,171148.00,171934.00,173426.00,167307.50
32808,87190.00,86861.00,86865.0,87359.0,88006.00,88790.00,89536.0,90645.0,91583.00,93203.00,94105.00,95815.00,89996.50
32824,193572.00,194299.00,194869.0,195651.0,196299.00,197279.00,198478.0,200201.0,202400.00,204284.00,205445.00,207293.00,199172.50


In [8]:
orlando_2015.index

Index([         32822,          32828,          32792,          32825,
                32839,          32837,          32835,          32808,
                32824,          32818,          32811,          32812,
                32807,          32817,          32806,          32819,
                32789,          32821,          32832,          32803,
                32801,          32809,          32836,          32804,
                32826,          32805,          32829,          32827,
                32814,          32831, '2015_average'],
      dtype='object', name='zipcode')

### Orlando 2016

In [9]:
orlando_2016 = city_orlando[['zipcode', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31',
       '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30',
       '2016-12-31']]

orlando_2016.set_index("zipcode", inplace=True)

orlando_2016 = orlando_2016.append(orlando_2016.mean().round(2).rename("2016_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2016.rename(columns={"average":"2016_average"}, inplace=True)
orlando_2016


,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2016_average
zipcode,,,,,,,,,,,,,
32822,131116.00,132619.0,133704.00,134293.00,135478.0,136593.00,137786.0,138973.0,140047.00,141424.00,142564.00,143483.0,137340.00
32828,252438.00,254479.0,255892.00,256070.00,257599.0,258881.00,260497.0,261811.0,262696.00,264235.00,265127.00,265847.0,259631.00
32792,196356.00,199179.0,202190.00,204429.00,205856.0,207218.00,208376.0,209966.0,211782.00,214634.00,216448.00,217421.0,207821.25
32825,194810.00,196820.0,198286.00,198916.00,200454.0,201992.00,203568.0,205045.0,206102.00,207538.00,208527.00,209242.0,202608.33
32839,122291.00,123367.0,124418.00,125348.00,126655.0,127781.00,129052.0,130278.0,131233.00,132430.00,133768.00,135092.0,128476.08
32837,217523.00,219704.0,221074.00,221553.00,222908.0,224205.00,225725.0,227003.0,227773.00,228972.00,229918.00,230576.0,224744.50
32835,174872.00,175955.0,176514.00,176805.00,177788.0,178700.00,179620.0,180553.0,181273.00,182313.00,183107.00,183772.0,179272.67
32808,97228.00,98930.0,99990.00,100507.00,101558.0,103027.00,104727.0,106558.0,108155.00,109557.00,110863.00,112134.0,104436.17
32824,209343.00,211159.0,212653.00,213464.00,215175.0,216700.00,218132.0,219398.0,220255.00,221606.00,222324.00,222902.0,216925.92


### Orlando 2017

In [10]:
orlando_2017 = city_orlando[['zipcode', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31',
       '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30',
       '2017-12-31']]

orlando_2017.set_index("zipcode", inplace=True)

orlando_2017 = orlando_2017.append(orlando_2017.mean().round(2).rename("2017_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2017.rename(columns={"average":"2017_average"}, inplace=True)
orlando_2017

,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2017_average
zipcode,,,,,,,,,,,,,
32822,145162.0,147097.00,149123.00,151149.0,152777.0,154811.0,156648.00,158493.00,160557.00,162214.00,164045.0,165607.00,155640.25
32828,267705.0,269984.00,272242.00,274135.0,275378.0,277521.0,279021.00,280535.00,282297.00,283608.00,285837.0,287550.00,277984.42
32792,217998.0,219226.00,221213.00,223882.0,226552.0,229112.0,231304.00,233558.00,235545.00,237052.00,239588.0,242641.00,229805.92
32825,210879.0,212668.00,214716.00,216712.0,218242.0,220349.0,222084.00,223790.00,225786.00,227464.00,229375.0,230983.00,221087.33
32839,137162.0,139072.00,141112.00,142553.0,143846.0,145642.0,147244.00,149006.00,150788.00,152463.00,154308.0,156015.00,146600.92
32837,232489.0,234526.00,236776.00,238652.0,239981.0,242046.0,243574.00,245231.00,247194.00,248570.00,250159.0,251359.00,242546.42
32835,185600.0,187447.00,189310.00,190707.0,191768.0,193410.0,194716.00,196067.00,197540.00,198718.00,199966.0,200933.00,193848.50
32808,113185.0,114628.00,116407.00,119262.0,121670.0,124021.0,126300.00,128113.00,130262.00,131964.00,133956.0,135789.00,124629.75
32824,224937.0,227360.00,229632.00,231247.0,232315.0,234139.0,235657.00,237137.00,238746.00,239806.00,241778.0,243601.00,234696.25


### Orlando 2018

In [11]:
orlando_2018 = city_orlando[['zipcode', '2018-01-31', '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31',
       '2018-06-30', '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30',
       '2018-12-31']]

orlando_2018.set_index("zipcode", inplace=True)

orlando_2018 = orlando_2018.append(orlando_2018.mean().round(2).rename("2018_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2018.rename(columns={"average":"2018_average"}, inplace=True)
orlando_2018

,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2018_average
zipcode,,,,,,,,,,,,,
32822,166127.00,167136.0,168525.0,171074.00,173312.00,175190.00,176956.00,178330.00,179498.00,180909.00,182706.00,184889.00,175387.67
32828,287877.00,288718.0,290248.0,293499.00,296016.00,297704.00,299449.00,300639.00,301941.00,303183.00,304613.00,306248.00,297511.25
32792,245221.00,246095.0,246253.0,246879.00,248814.00,251239.00,253637.00,255418.00,256719.00,258175.00,258929.00,260062.00,252286.75
32825,231282.00,232538.0,234137.0,237121.00,239518.00,241187.00,242834.00,244076.00,245430.00,246849.00,248427.00,250125.00,241127.00
32839,156473.00,157502.0,158787.0,161646.00,164359.00,166909.00,169366.00,171154.00,172988.00,174871.00,176857.00,178755.00,167472.25
32837,251547.00,252480.0,253929.0,256598.00,258761.00,260119.00,261637.00,262700.00,263932.00,265352.00,267298.00,269330.00,260306.92
32835,200922.00,201794.0,203052.0,205471.00,207440.00,208887.00,210414.00,211393.00,212694.00,214094.00,215843.00,217648.00,209137.67
32808,137333.00,139203.0,141214.0,143644.00,145913.00,147827.00,149597.00,151073.00,152591.00,154274.00,156277.00,158228.00,148097.83
32824,243978.00,244751.0,245958.0,248876.00,251234.00,252795.00,254434.00,255582.00,257071.00,258752.00,260749.00,262506.00,253057.17


### Orlando 2019

In [12]:
orlando_2019 = city_orlando[['zipcode', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31',
       '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31', '2019-11-30',
       '2019-12-31']]

orlando_2019.set_index("zipcode", inplace=True)

orlando_2019 = orlando_2019.append(orlando_2019.mean().round(2).rename("2019_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2019.rename(columns={"average":"2019_average"}, inplace=True)
orlando_2019

,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2019_average
zipcode,,,,,,,,,,,,,
32822,187101.00,188363.00,189234.0,189168.00,189594.00,190036.00,191039.00,191921.00,193126.0,193964.0,195035.00,195907.00,191207.33
32828,307817.00,308427.00,308617.0,307803.00,307525.00,306982.00,307120.00,307313.00,308082.0,308429.0,309012.00,309886.00,308084.42
32792,261694.00,263858.00,265269.0,265452.00,265813.00,266153.00,266890.00,267246.00,268182.0,268759.0,269157.00,269689.00,266513.50
32825,251951.00,252735.00,253311.0,252800.00,252978.00,253035.00,253727.00,254453.00,255498.0,256119.0,257024.00,258084.00,254309.58
32839,180952.00,182277.00,183379.0,183522.00,184091.00,184185.00,184520.00,185047.00,185959.0,186568.0,187405.00,188332.00,184686.42
32837,271142.00,271739.00,272232.0,271790.00,271829.00,271715.00,272178.00,272743.00,273781.0,274487.0,275291.00,276115.00,272920.17
32835,219380.00,220229.00,220854.0,220526.00,220832.00,220966.00,221705.00,222444.00,223420.0,224068.0,224747.00,225359.00,222044.17
32808,160554.00,161809.00,162886.0,162727.00,163030.00,162940.00,163323.00,163921.00,165254.0,166381.0,167629.00,168511.00,164080.42
32824,263996.00,264435.00,264925.0,264629.00,264908.00,265129.00,265826.00,266664.00,267857.0,268572.0,269190.00,269753.00,266323.67


### Orlando 2020

In [13]:
orlando_2020 = city_orlando[['zipcode', '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30', '2020-05-31',
       '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
       '2020-12-31']]

orlando_2020.set_index("zipcode", inplace=True)

orlando_2020 = orlando_2020.append(orlando_2020.mean().round(2).rename("2020_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2020.rename(columns={"average":"2020_average"}, inplace=True)
orlando_2020

,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2020_average
zipcode,,,,,,,,,,,,,
32822,197073.0,198808.00,200393.0,202249.00,203616.0,204570.00,205293.00,206440.0,207860.00,209668.0,211229.0,213046.00,205020.42
32828,311080.0,313118.00,314686.0,316934.00,318789.0,320290.00,321480.00,323126.0,325149.00,327891.0,330530.0,333372.00,321370.42
32792,270854.0,273259.00,275829.0,279268.00,281358.0,282002.00,282049.00,282948.0,284987.00,287539.0,290751.0,293997.00,282070.08
32825,259475.0,261441.00,263050.0,265195.00,266759.0,267912.00,268718.00,269961.0,271575.00,273765.0,275891.0,278249.00,268499.25
32839,190078.0,192187.00,194154.0,195737.00,196965.0,197659.00,198380.00,199214.0,200477.00,202093.0,203596.0,205351.00,197990.92
32837,277282.0,279148.00,280365.0,282173.00,283597.0,284610.00,285258.00,286243.0,287535.00,289335.0,291057.0,293180.00,284981.92
32835,226228.0,227606.00,228802.0,230519.00,232016.0,232990.00,233558.00,234308.0,235396.00,236877.0,238322.0,240080.00,233058.50
32808,169916.0,171851.00,173842.0,175720.00,177129.0,178425.00,179696.00,181141.0,182328.00,183811.0,185080.0,186920.00,178821.58
32824,270792.0,272546.00,273830.0,275604.00,277286.0,278402.00,279186.00,280039.0,281323.00,283190.0,285048.0,287418.00,278722.00


### Orlando 2021 (until June)

In [14]:
orlando_2021 = city_orlando[['zipcode','2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31',
       '2021-06-30']]

orlando_2021.set_index("zipcode", inplace=True)

orlando_2021 = orlando_2021.append(orlando_2021.mean().round(2).rename("2021_average")).assign(average = lambda d: d.mean(1).round(2))

orlando_2021.rename(columns={"average":"2021_average"}, inplace=True)
orlando_2021

,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021_average
zipcode,,,,,,,
32822,215063.00,217226.00,219868.00,222782.00,226231.00,230418.00,221931.33
32828,336709.00,340177.00,344563.00,349060.00,354359.00,360861.00,347621.50
32792,297043.00,299631.00,303114.00,306945.00,312076.00,318976.00,306297.50
32825,280900.00,283755.00,287298.00,291072.00,295576.00,301136.00,289956.17
32839,206813.00,208559.00,210735.00,213649.00,216987.00,221388.00,213021.83
32837,295710.00,298512.00,302159.00,305981.00,310475.00,316072.00,304818.17
32835,242083.00,244157.00,246636.00,249201.00,252022.00,255773.00,248312.00
32808,188390.00,190079.00,191939.00,194598.00,197664.00,201369.00,194006.50
32824,290152.00,293084.00,296627.00,300205.00,304138.00,309195.00,298900.17


### Merge average price data

In [15]:
average = [orlando_2015['2015_average'], orlando_2016['2016_average'], orlando_2017['2017_average'], orlando_2018['2018_average'], orlando_2019['2019_average'], orlando_2020['2020_average'], orlando_2021['2021_average']]

headers = ['2015','2016','2017','2018','2019','2020','2020']

average_df = pd.concat(average, axis=1, keys=headers)

n = 7
average_df.drop(average_df.tail(n).index, inplace=True)

# average_df['avg'] = average_df[headers].agg("mean")

average_df

,2015,2016,2017,2018,2019,2020,2020
zipcode,,,,,,,
32822,123245.58,137340.00,155640.25,175387.67,191207.33,205020.42,221931.33
32828,242952.83,259631.00,277984.42,297511.25,308084.42,321370.42,347621.50
32792,186618.00,207821.25,229805.92,252286.75,266513.50,282070.08,306297.50
32825,184553.83,202608.33,221087.33,241127.00,254309.58,268499.25,289956.17
32839,117559.92,128476.08,146600.92,167472.25,184686.42,197990.92,213021.83
32837,207999.58,224744.50,242546.42,260306.92,272920.17,284981.92,304818.17
32835,167307.50,179272.67,193848.50,209137.67,222044.17,233058.50,248312.00
32808,89996.50,104436.17,124629.75,148097.83,164080.42,178821.58,194006.50
32824,199172.50,216925.92,234696.25,253057.17,266323.67,278722.00,298900.17


In [16]:
orlando_df = average_df.mean(axis='columns').round(2).reset_index().rename(columns={0:"price"})
orlando_df

,zipcode,price
0,32822,172824.65
1,32828,293593.69
2,32792,247344.71
3,32825,237448.78
4,32839,165115.48
5,32837,256902.53
6,32835,207568.72
7,32808,143438.39
8,32824,249685.38
9,32818,201290.52


In [17]:
nomi = pgeocode.Nominatim('us')
df_1 = nomi.query_postal_code(['32801','32804','32789','32811','32835','32819','32839','32824','32827','32822','32829','32832','32808','32817','32828','32818','32821','32825','32826','32836','32837','32805','32806','32803','32814','32809','32831','32792','32812','32807'])

In [18]:
df_coor = df_1[['postal_code','latitude','longitude']]
df_coor.rename(columns={"postal_code":"zipcode"}, inplace=True)
df_coor

C:\Users\aliso\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,zipcode,latitude,longitude
0,32801,28.5399,-81.3727
1,32804,28.5754,-81.3955
2,32789,28.5978,-81.3534
3,32811,28.5163,-81.4516
4,32835,28.5289,-81.4787
5,32819,28.4522,-81.4678
6,32839,28.4871,-81.4082
7,32824,28.3932,-81.3622
8,32827,28.4317,-81.3430
9,32822,28.4944,-81.2902


In [19]:
df_coor.zipcode.unique()

array(['32801', '32804', '32789', '32811', '32835', '32819', '32839',
       '32824', '32827', '32822', '32829', '32832', '32808', '32817',
       '32828', '32818', '32821', '32825', '32826', '32836', '32837',
       '32805', '32806', '32803', '32814', '32809', '32831', '32792',
       '32812', '32807'], dtype=object)

## Merge datasets

In [20]:
df_merged = orlando_df[['zipcode','price']]
df_1 = df_merged.sort_values(by=['zipcode'])

df_2 = df_coor.sort_values(by=['zipcode'])
df_2 = df_2.loc[df_2["zipcode"].isin(['32801','32804','32789','32811','32835','32819','32839','32824','32827','32822','32829','32832','32808','32817','32828','32818','32821','32825','32826','32836','32837','32805','32806','32803','32814','32809','32831','32792','32812','32807'])]
df_2.reset_index(drop=True, inplace=True)
df_1.reset_index(drop=True, inplace=True)

df_2['price'] = df_1['price']
# df_2.zipcode.unique()

In [21]:
df_2.to_csv('data/home_value.csv', index=False)

In [22]:
df_1

,zipcode,price
0,32789,480361.71
1,32792,247344.71
2,32801,282031.02
3,32803,299539.95
4,32804,329537.92
5,32805,129388.47
6,32806,282839.77
7,32807,188152.37
8,32808,143438.39
9,32809,186789.79


In [23]:
df_2

,zipcode,latitude,longitude,price
0,32789,28.5978,-81.3534,480361.71
1,32792,28.5974,-81.3036,247344.71
2,32801,28.5399,-81.3727,282031.02
3,32803,28.5559,-81.3535,299539.95
4,32804,28.5754,-81.3955,329537.92
5,32805,28.5302,-81.4045,129388.47
6,32806,28.5140,-81.3570,282839.77
7,32807,28.5515,-81.3051,188152.37
8,32808,28.5803,-81.4396,143438.39
9,32809,28.4637,-81.3948,186789.79


In [24]:
df_1.zipcode.unique()

array([32789, 32792, 32801, 32803, 32804, 32805, 32806, 32807, 32808,
       32809, 32811, 32812, 32814, 32817, 32818, 32819, 32821, 32822,
       32824, 32825, 32826, 32827, 32828, 32829, 32831, 32832, 32835,
       32836, 32837, 32839], dtype=int64)

In [25]:
df_2.zipcode.unique()

array(['32789', '32792', '32801', '32803', '32804', '32805', '32806',
       '32807', '32808', '32809', '32811', '32812', '32814', '32817',
       '32818', '32819', '32821', '32822', '32824', '32825', '32826',
       '32827', '32828', '32829', '32831', '32832', '32835', '32836',
       '32837', '32839'], dtype=object)